# PRV Assignment 3

- Pablo Garcia Fernández

In [9]:
# Initial configuration
import open3d as o3d
import numpy as np
import copy
import time

## Registration methodology

Since the ICP registration algorithm needs an initial transformation that roughly aligns the source point cloud to the target point cloud (and a poor initial allignment might fail ICP convergence), the first step is to perform an initial guess through  global registration.

Global registration methods do not require an alignment for initialization, but they usually produce poorer results. For this reason they are used as initialization of the local methods (like ICP). To perform the global registration we first downsample the point cloud using voxelization methods to structure the point cloud in 3D layers. Then, we estimate normals; and finally we compute FPFH feature for each point. This features are used (as in image alignment methods) to guess salient points and find correspondences between clouds.

There are many features descriptors that can be used to describe point in clouds. Here we choose FPFH (Fast Point Feature Histograms), which is a robust multi-dimensional features which describe the local geometry around a point p for 3D point cloud *(R. B. Rusu, N. Blodow and M. Beetz, "Fast Point Feature Histograms (FPFH) for 3D registration," 2009 IEEE International Conference on Robotics and Automation, 2009, pp. 3212-3217, doi: 10.1109/ROBOT.2009.5152473)* FPFH represent the underlying surface model properties at a point p through the combination of certain geometrical relations between p’s nearest k neighbors. The final output (for each point p) is a 33-dimensional vector that describes the local geometric property of p.


Once the features are computed, RANSAC is used for global registration. In each RANSAC iteration, some points are picked from the source. Their corresponding points in the target point cloud are detected by querying the nearest neighbor in the 33-dimensional FPFH feature space. Finally a prune step is applied. The matches that pass the pruning step are used to compute the transformation between the clouds.

As we have said, this initial transformation is not tight, so a local refinement must be done with ICP (Iterative Closest Point). Different ICP parameters and flavours (PointToPoint, PointToPlane) are tested to see its effect on the final alignment.

In [10]:
# Main functions to perform the registration.
# Some of these functions are based on open3D docs: 
# http://www.open3d.org/docs/latest/tutorial/Basic/icp_registration.html

def draw_registration_result(source, target, transformation):
    """
    Visualizes a target point cloud and a source point cloud transformed with an alignment transformation. 
    The target point cloud and the source point cloud are painted with cyan and yellow colors respectively. 
    """

    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])
                                      

def preprocess_point_cloud(pcd, voxel_size):
    """
    Downsample the point cloud, estimate normals, then compute a FPFH feature for each point. 
    The FPFH feature is a 33-dimensional vector that describes the local geometric property of a point.
    
    > R. Rusu, N. Blodow, and M. Beetz, Fast Point Feature Histograms (FPFH) for 3D registration, ICRA, 2009.
    """

    print("Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print("Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print("Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh


def prepare_dataset(voxel_size, point_c1, point_c2, draw=False):
    """
    Read a source point cloud and a target point cloud from two files. They are misaligned.
    Estimate normals (needed for ICP point to plane)
    Compute features.
    """
    print("Load two point clouds.")
    source = o3d.io.read_point_cloud(point_c1)
    target = o3d.io.read_point_cloud(point_c2)
    
    print("Estimate normals")
    source.estimate_normals()
    target.estimate_normals()

    if draw:
        print("Initial misaligned:")
        draw_registration_result(source, target, np.identity(4))  # Show initial misaligned

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh


def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    """
    Global registration
    """
    distance_threshold = voxel_size * 1.5
    print("RANSAC registration on downsampled point clouds.")
    print("Since the downsampling voxel size is %.3f," % voxel_size)
    print("we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True, distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        4, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 0.999))
    return result


def refine_registration(source, target, result_ransac, voxel_size, method=o3d.pipelines.registration.TransformationEstimationPointToPlane()):
    """
    ICP registration. The input are two point clouds and an initial transformation (global registration result) that roughly 
    aligns the source point cloud to the target point cloud.
    
    We will try 2 main methods: PointToPoint and PointToPlane with different parameters
    """
    
    distance_threshold = voxel_size * 0.4
    print("Point-to-plane ICP registration is applied on original point")
    print("clouds to refine the alignment. This time we use a strict")
    print("distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        method)
    return result

## Dataset 1

First, using Cloud Compare we obtain the ground truth of the alignment. It will be used for evaluation purposes. To evaluate the quality of the computed registration transformation, we will used the distance (Frobenius norm) between the ground truth and the computed transformations. The lower the value, the better.

$$\left[
\begin{array}{ll}
0.996401 & 0.083261 & 0.015885 & -1.239532 \\
-0.083465 & 0.996431 & 0.012612 & 0.060150 \\
-0.014778 & -0.013892 & 0.999795 & 0.013994 \\
0.000000 & 0.000000 & 0.000000 & 1.000000 \\
\end{array}
\right]$$


*Note: we include some images as attachments because open3d display methods do not allow us to show the result in the notebook.*

### Experiment 1: Baseline

We perform the registration with default good parameters to obtain a baseline for comparation

In [11]:
# Configuration
point_cloud1_1 = "./data1/000009.pcd"
point_cloud1_2 = "./data1/000013.pcd"

gt_transf = np.array([
            [0.996401, 0.083261, 0.015885, -1.239532],
            [-0.083465, 0.996431, 0.012612, 0.060150],
            [-0.014778, -0.013892, 0.999795, 0.013994],
            [0.000000, 0.000000, 0.000000, 1.000000]
])


Prepare dataset, compute features and show initial misaligned:

In [12]:
voxel_size = 0.1
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size, point_cloud1_1, point_cloud1_2, True)

# Compute initial alignmetn quality
print("\n!! Distance:", np.linalg.norm(gt_transf - np.identity(4)))

Load two point clouds.
Estimate normals
Initial misaligned:
Downsample with a voxel size 0.100.
Estimate normal with search radius 0.200.
Compute FPFH feature with search radius 0.500.
Downsample with a voxel size 0.100.
Estimate normal with search radius 0.200.
Compute FPFH feature with search radius 0.500.

!! Distance: 1.2469966877301641


We can clearly see the misalignment, for example on the poster.

<img src="./img/exp1_misalign.PNG" width="800">

**Perform global registration**

In [13]:
start = time.time()
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
fin = time.time()
draw_registration_result(source_down, target_down, result_ransac.transformation)

print(f"\nTransformation:\n{result_ransac.transformation}\n")
print("!! Distance:", np.linalg.norm(gt_transf - result_ransac.transformation))
print("Execution time: ", fin-start)

RANSAC registration on downsampled point clouds.
Since the downsampling voxel size is 0.100,
we use a liberal distance threshold 0.150.

Transformation:
[[ 0.99624598  0.08456448  0.01851457 -1.31173011]
 [-0.08487046  0.99625677  0.01641519  0.04458664]
 [-0.01705712 -0.01792491  0.99969383  0.05159821]
 [ 0.          0.          0.          1.        ]]

!! Distance: 0.08315911675431552
Execution time:  3.080801248550415


The result has improved. Visually it can be seen in the post. Numerically in the decrease of the distance between the transformations.

<img src="./img/exp1_global.PNG" width="800">

**Perform ICP refinement**

In [14]:
start = time.time()
result_icp = refine_registration(source, target, result_ransac, voxel_size, method=o3d.pipelines.registration.TransformationEstimationPointToPoint())
fin = time.time()
draw_registration_result(source, target, result_icp.transformation)

print(f"\nTransformation:\n{result_icp.transformation}\n")
print("!! Distance:", np.linalg.norm(gt_transf - result_icp.transformation))
print("Execution time: ", fin-start)

Point-to-plane ICP registration is applied on original point
clouds to refine the alignment. This time we use a strict
distance threshold 0.040.

Transformation:
[[ 0.99632099  0.08366965  0.01854372 -1.31074271]
 [-0.08394143  0.99636669  0.01439606  0.01377643]
 [-0.01727184 -0.01589968  0.9997244   0.00972672]
 [ 0.          0.          0.          1.        ]]

!! Distance: 0.08520912017003214
Execution time:  0.3715336322784424


The result has improved (distance decreasing).

<img src="./img/exp1_icp.PNG" width="800">

### Experiment 2: Voxel size effect

Voxel size if a very important parameter when downsampling the point clouds. Lower voxel sizes are equivalent to higher resolution, while larger voxel sizes imply lower resolution. 

When performing global registration method, high-resolution downsampling (or equivalently small voxel sizes) leads to longer execution time and the possibility of falling into local minimums. 

This is what happens in the following example: the global registration time increase up to 16s, but transformation does not change.

In this experiment we can also observed how a bad initial transformation guess for the ICP method, leads the algorithm not to converge and consequently to obtain a bad alignment. In this case the final transformation is almost the identity matrix (no minor differences with respect the initial positions).

In [15]:
# EXPERIMENT 2 -> with lowe voxel size
voxel_size = 0.01
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size, point_cloud1_1, point_cloud1_2)

# Compute initial alignmetn quality
print("\n!! Initial distance:", np.linalg.norm(gt_transf - np.identity(4)), "\n\n")

print("> GLOBAL REGISTRATION!!!")
start = time.time()
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
fin = time.time()
print(f"\nTransformation:\n{result_ransac.transformation}")
print("!! Distance:", np.linalg.norm(gt_transf - result_ransac.transformation))
print("Global registration time: ", fin-start, "\n\n")

print("> ICP!!!")
start = time.time()
result_icp = refine_registration(source, target, result_ransac, voxel_size, method=o3d.pipelines.registration.TransformationEstimationPointToPoint())
fin = time.time()

print(f"Transformation:\n{result_icp.transformation}")
print("!! Distance:", np.linalg.norm(gt_transf - result_icp.transformation))
print("ICP time: ", fin-start)


Load two point clouds.
Estimate normals
Downsample with a voxel size 0.010.
Estimate normal with search radius 0.020.
Compute FPFH feature with search radius 0.050.
Downsample with a voxel size 0.010.
Estimate normal with search radius 0.020.
Compute FPFH feature with search radius 0.050.

!! Initial distance: 1.2469966877301641 


> GLOBAL REGISTRATION!!!
RANSAC registration on downsampled point clouds.
Since the downsampling voxel size is 0.010,
we use a liberal distance threshold 0.015.

Transformation:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
!! Distance: 1.2469966877301641
Global registration time:  16.205626010894775 


> ICP!!!
Point-to-plane ICP registration is applied on original point
clouds to refine the alignment. This time we use a strict
distance threshold 0.004.
Transformation:
[[ 9.99999231e-01 -3.26017659e-04  1.19629756e-03 -5.18689945e-04]
 [ 3.25145636e-04  9.99999681e-01  7.29056972e-04 -9.74497241e-04]
 [-1.19653486e-03 -7.28667441e-04  9.99999

### EXPERIMENT 3: ICP PointToPlane

In the previous examples we were using the Point-To-Point ICP registration, where we first show a point-to-point ICP algorithm using the objective:

$$E(T)=∑_{(p,q)∈K}∥p−T_q∥^2$$

However, there is also another variant called Point-to-Plane that uses a different objective function:

$$E(T)=∑(_{p,q)∈K}((p−T_q)⋅n_p)^2$$

where n_p is the normal of point p. Equations extracted from http://www.open3d.org/docs/latest/tutorial/t_pipelines/t_icp_registration.html

In this example we compare this alternative version with the baseline (experiment 1):

- As we do not have the normals in the .pcd file, we first estimate them in the `prepare_dataset()` function.
- The final distance shows how this method achieve a better registration result (less difference between the ground truth and the obtained transofmations) in a shoter time.


In [16]:
# EXPERIMENT 3 -> Point-To-Plane with good voxel size
voxel_size = 0.1
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size, point_cloud1_1, point_cloud1_2)

# Compute initial alignmetn quality
print("\n!! Initial distance:", np.linalg.norm(gt_transf - np.identity(4)), "\n\n")

print("> GLOBAL REGISTRATION!!!")
start = time.time()
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
fin = time.time()
print(f"\nTransformation:\n{result_ransac.transformation}")
print("!! Distance:", np.linalg.norm(gt_transf - result_ransac.transformation))
print("Global registration time: ", fin-start, "\n\n")

print("> ICP!!!")
start = time.time()
result_icp = refine_registration(source, target, result_ransac, voxel_size, method=o3d.pipelines.registration.TransformationEstimationPointToPlane())  ### here!!!
fin = time.time()

print(f"Transformation:\n{result_icp.transformation}")
print("!! Distance:", np.linalg.norm(gt_transf - result_icp.transformation))
print("ICP time: ", fin-start)

Load two point clouds.
Estimate normals
Downsample with a voxel size 0.100.
Estimate normal with search radius 0.200.
Compute FPFH feature with search radius 0.500.
Downsample with a voxel size 0.100.
Estimate normal with search radius 0.200.
Compute FPFH feature with search radius 0.500.

!! Initial distance: 1.2469966877301641 


> GLOBAL REGISTRATION!!!
RANSAC registration on downsampled point clouds.
Since the downsampling voxel size is 0.100,
we use a liberal distance threshold 0.150.

Transformation:
[[ 0.99605972  0.08640227  0.01999185 -1.29694619]
 [-0.08663592  0.99617786  0.01113043  0.00573418]
 [-0.01895374 -0.01281858  0.99973819  0.0356652 ]
 [ 0.          0.          0.          1.        ]]
!! Distance: 0.08237030998686633
Global registration time:  3.119835376739502 


> ICP!!!
Point-to-plane ICP registration is applied on original point
clouds to refine the alignment. This time we use a strict
distance threshold 0.040.
Transformation:
[[ 0.9963475   0.08336077  0.018

## DATASET 2

Falta ejecutar lo mismo que antes (sin experimentos solo base) con el dataset de clase

## CONCLUSIONS

- Point-To-Plane mejor que PtP tiempo
- Importancia voxel size
- Importancia ajuste inicial ICP